In [5]:
import pandas as pd
import numpy as np

import pytz
import astral as ast
from astral import LocationInfo
from astral.sun import sun
from datetime import datetime, timedelta, date

In [7]:
# read in with pandas
winter1 = pd.read_csv ('jhw20-21data.txt')
winter2 = pd.read_csv ('jhw21-22data.txt')
winter3 = pd.read_csv ('jhw22-23data.txt')

In [9]:
# rename columns
winter1 = winter1.rename(columns = {'valid': 'datetime', 'p01m': 'precip(mm)'})
winter2 = winter2.rename(columns = {'valid': 'datetime', 'p01m': 'precip(mm)'})
winter3 = winter3.rename(columns = {'valid': 'datetime', 'p01m': 'precip(mm)'})

# datetime format
winter1['datetime'] = pd.to_datetime(winter1['datetime'])
winter2['datetime'] = pd.to_datetime(winter2['datetime'])
winter3['datetime'] = pd.to_datetime(winter3['datetime'])

In [11]:
# mask out days without precipitation
codes = ['GR','GS','IC','PL','PY','RA','SG','SH','SN','SQ','TS','UP']

mask1 = winter1['wxcodes'].apply(lambda x: any(sub in x for sub in codes))
mask2 = winter2['wxcodes'].apply(lambda x: any(sub in x for sub in codes))
mask3 = winter3['wxcodes'].apply(lambda x: any(sub in x for sub in codes))

precip_days1 = winter1[mask1]
precip_days2 = winter2[mask2]
precip_days3 = winter3[mask3]

In [13]:
precip_days2

,station,datetime,precip(mm),wxcodes
0,JHW,2021-12-01 00:13:00,T,-RA
1,JHW,2021-12-01 00:44:00,T,UP
3,JHW,2021-12-01 01:21:00,T,-SN
4,JHW,2021-12-01 01:56:00,T,-RA
6,JHW,2021-12-01 03:27:00,T,-SN
...,...,...,...,...
4037,JHW,2022-02-27 21:56:00,T,-SN
4038,JHW,2022-02-27 22:56:00,T,-SN
4039,JHW,2022-02-27 23:40:00,T,-SN
4040,JHW,2022-02-27 23:48:00,T,-SN


In [15]:
# replace Trace with 0.0001
precip_days1['precip(mm)'] = precip_days1['precip(mm)'].replace('T', 0.0001)
precip_days2['precip(mm)'] = precip_days2['precip(mm)'].replace('T', 0.0001)
precip_days3['precip(mm)'] = precip_days3['precip(mm)'].replace('T', 0.0001)

C:\Users\alyha\AppData\Local\Temp\ipykernel_30052\3060002225.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days1['precip(mm)'] = precip_days1['precip(mm)'].replace('T', 0.0001)
C:\Users\alyha\AppData\Local\Temp\ipykernel_30052\3060002225.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days2['precip(mm)'] = precip_days2['precip(mm)'].replace('T', 0.0001)
C:\Users\alyha\AppData\Local\Temp\ipykernel_30052\3060002225.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of

In [17]:
# make sure numeric form
precip_days1['precip(mm)'] = pd.to_numeric(precip_days1['precip(mm)'])
precip_days2['precip(mm)'] = pd.to_numeric(precip_days2['precip(mm)'])
precip_days3['precip(mm)'] = pd.to_numeric(precip_days3['precip(mm)'])

C:\Users\alyha\AppData\Local\Temp\ipykernel_30052\2630444437.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days1['precip(mm)'] = pd.to_numeric(precip_days1['precip(mm)'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_30052\2630444437.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days2['precip(mm)'] = pd.to_numeric(precip_days2['precip(mm)'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_30052\2630444437.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fro

In [19]:
# eliminate special weather 
precip_days1 = precip_days1[precip_days1['datetime'].dt.minute == 56]
precip_days2 = precip_days2[precip_days2['datetime'].dt.minute == 56]
precip_days3 = precip_days3[precip_days3['datetime'].dt.minute == 56]

In [21]:
precip_days1.dtypes

station               object
datetime      datetime64[ns]
precip(mm)           float64
wxcodes               object
dtype: object

In [23]:
precip_days1['date'] = precip_days1['datetime'].dt.date

In [26]:
jhw = LocationInfo(
    'Jamestown', 'New York', 
    latitude=42.0970, 
    longitude=-79.2353, 
    timezone='America/New_York'
)

jhw

LocationInfo(name='Jamestown', region='New York', timezone='America/New_York', latitude=42.097, longitude=-79.2353)

In [28]:
# create a date range (winter 1)
date_range1 = pd.date_range(start='2020-12-01', end='2021-2-28', tz='America/New_York')

# lists to store sunrise and sunset times
sunrise_times1 = []
sunset_times1 = []

# Calculate sunrise and sunset for each date
for date in date_range1:
    s = sun(jhw.observer, date=date)
    
    sunrise_times1.append(s['sunrise'])
    sunset_times1.append(s['sunset'])

# df with results
sun_times1 = pd.DataFrame({
    'datetime': date_range1.date,
    'sunrise': sunrise_times1,
    'sunset': sunset_times1
})

sun_times1['sunrise'] = pd.to_datetime(sun_times1['sunrise'])
sun_times1['sunset'] = pd.to_datetime(sun_times1['sunset'])

In [30]:
# create a date range (winter 2)
date_range2 = pd.date_range(start='2021-12-01', end='2022-2-28', tz='America/New_York')

# lists to store sunrise and sunset times
sunrise_times2 = []
sunset_times2 = []

# Calculate sunrise and sunset for each date
for date in date_range2:
    s = sun(jhw.observer, date=date)
    
    sunrise_times2.append(s['sunrise'])
    sunset_times2.append(s['sunset'])

# df with results
sun_times2 = pd.DataFrame({
    'datetime': date_range2.date,
    'sunrise': sunrise_times2,
    'sunset': sunset_times2
})

sun_times2['sunrise'] = pd.to_datetime(sun_times2['sunrise'])
sun_times2['sunset'] = pd.to_datetime(sun_times2['sunset'])

In [32]:
# create a date range (winter 3)
date_range3 = pd.date_range(start='2022-12-01', end='2023-2-28', tz='America/New_York')

# lists to store sunrise and sunset times
sunrise_times3 = []
sunset_times3 = []

# Calculate sunrise and sunset for each date
for date in date_range3:
    s = sun(jhw.observer, date=date)
    
    sunrise_times3.append(s['sunrise'])
    sunset_times3.append(s['sunset'])

# df with results
sun_times3 = pd.DataFrame({
    'datetime': date_range3.date,
    'sunrise': sunrise_times3,
    'sunset': sunset_times3
})

sun_times3['sunrise'] = pd.to_datetime(sun_times3['sunrise'])
sun_times3['sunset'] = pd.to_datetime(sun_times3['sunset'])

In [34]:
# add sunrise and sunset times (winter1)
sun_times1['sunrise'] = sun_times1['sunrise'].dt.strftime('%Y-%m-%d %H:%M:%S')
sun_times1['sunset'] = sun_times1['sunset'].dt.strftime('%Y-%m-%d %H:%M:%S')

sun_times1['date'] = sun_times1['datetime']
sun_times1 = sun_times1.drop(columns=['datetime'])

print(sun_times1)

                sunrise               sunset        date
0   2020-12-01 07:26:30  2020-12-01 16:45:42  2020-12-01
1   2020-12-02 07:27:32  2020-12-02 16:45:27  2020-12-02
2   2020-12-03 07:28:33  2020-12-03 16:45:14  2020-12-03
3   2020-12-04 07:29:33  2020-12-04 16:45:04  2020-12-04
4   2020-12-05 07:30:32  2020-12-05 16:44:56  2020-12-05
..                  ...                  ...         ...
85  2021-02-24 06:59:41  2021-02-24 18:01:04  2021-02-24
86  2021-02-25 06:58:08  2021-02-25 18:02:18  2021-02-25
87  2021-02-26 06:56:35  2021-02-26 18:03:31  2021-02-26
88  2021-02-27 06:55:01  2021-02-27 18:04:44  2021-02-27
89  2021-02-28 06:53:26  2021-02-28 18:05:56  2021-02-28

[90 rows x 3 columns]


In [36]:
# add sunrise and sunset times (winter2)

sun_times2['sunrise'] = sun_times2['sunrise'].dt.strftime('%Y-%m-%d %H:%M:%S')
sun_times2['sunset'] = sun_times2['sunset'].dt.strftime('%Y-%m-%d %H:%M:%S')

sun_times2['date'] = sun_times2['datetime']
sun_times2 = sun_times2.drop(columns=['datetime'])

print(sun_times2)

                sunrise               sunset        date
0   2021-12-01 07:26:15  2021-12-01 16:45:45  2021-12-01
1   2021-12-02 07:27:17  2021-12-02 16:45:30  2021-12-02
2   2021-12-03 07:28:19  2021-12-03 16:45:17  2021-12-03
3   2021-12-04 07:29:19  2021-12-04 16:45:06  2021-12-04
4   2021-12-05 07:30:18  2021-12-05 16:44:57  2021-12-05
..                  ...                  ...         ...
85  2022-02-24 07:00:03  2022-02-24 18:00:46  2022-02-24
86  2022-02-25 06:58:31  2022-02-25 18:02:00  2022-02-25
87  2022-02-26 06:56:58  2022-02-26 18:03:13  2022-02-26
88  2022-02-27 06:55:24  2022-02-27 18:04:26  2022-02-27
89  2022-02-28 06:53:49  2022-02-28 18:05:39  2022-02-28

[90 rows x 3 columns]


In [38]:
# add sunrise and sunset times (winter3)

sun_times3['sunrise'] = sun_times3['sunrise'].dt.strftime('%Y-%m-%d %H:%M:%S')
sun_times3['sunset'] = sun_times3['sunset'].dt.strftime('%Y-%m-%d %H:%M:%S')

sun_times3['date'] = sun_times3['datetime']
sun_times3 = sun_times3.drop(columns=['datetime'])

print(sun_times3)

                sunrise               sunset        date
0   2022-12-01 07:25:59  2022-12-01 16:45:49  2022-12-01
1   2022-12-02 07:27:02  2022-12-02 16:45:33  2022-12-02
2   2022-12-03 07:28:04  2022-12-03 16:45:19  2022-12-03
3   2022-12-04 07:29:05  2022-12-04 16:45:08  2022-12-04
4   2022-12-05 07:30:04  2022-12-05 16:44:59  2022-12-05
..                  ...                  ...         ...
85  2023-02-24 07:00:25  2023-02-24 18:00:28  2023-02-24
86  2023-02-25 06:58:53  2023-02-25 18:01:42  2023-02-25
87  2023-02-26 06:57:20  2023-02-26 18:02:55  2023-02-26
88  2023-02-27 06:55:46  2023-02-27 18:04:08  2023-02-27
89  2023-02-28 06:54:12  2023-02-28 18:05:21  2023-02-28

[90 rows x 3 columns]


In [40]:
# add 'date' column to all dataframes
precip_days1['date'] = precip_days1['datetime'].dt.date
precip_days2['date'] = precip_days2['datetime'].dt.date
precip_days3['date'] = precip_days3['datetime'].dt.date

C:\Users\alyha\AppData\Local\Temp\ipykernel_30052\762931898.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days2['date'] = precip_days2['datetime'].dt.date


In [42]:
# merge precip days and sunrise/sunset times on date 
merged1 = pd.merge(precip_days1, sun_times1, on='date')

merged1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,JHW,2020-12-01 00:56:00,0.0001,UP BR,2020-12-01,2020-12-01 07:26:30,2020-12-01 16:45:42
1,JHW,2020-12-01 06:56:00,0.0001,UP,2020-12-01,2020-12-01 07:26:30,2020-12-01 16:45:42
2,JHW,2020-12-01 07:56:00,0.0001,-SN BR,2020-12-01,2020-12-01 07:26:30,2020-12-01 16:45:42
3,JHW,2020-12-01 08:56:00,0.0001,-SN BR,2020-12-01,2020-12-01 07:26:30,2020-12-01 16:45:42
4,JHW,2020-12-01 09:56:00,0.0001,-SN,2020-12-01,2020-12-01 07:26:30,2020-12-01 16:45:42
...,...,...,...,...,...,...,...
816,JHW,2021-02-26 05:56:00,0.0001,-SN BR,2021-02-26,2021-02-26 06:56:35,2021-02-26 18:03:31
817,JHW,2021-02-27 10:56:00,0.0001,UP,2021-02-27,2021-02-27 06:55:01,2021-02-27 18:04:44
818,JHW,2021-02-27 12:56:00,0.0001,-RA,2021-02-27,2021-02-27 06:55:01,2021-02-27 18:04:44
819,JHW,2021-02-27 13:56:00,2.7900,-RA,2021-02-27,2021-02-27 06:55:01,2021-02-27 18:04:44


In [44]:
# merge precip days and sunrise/sunset times on date 
merged2 = pd.merge(precip_days2, sun_times2, on='date')

merged2

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,JHW,2021-12-01 01:56:00,0.0001,-RA,2021-12-01,2021-12-01 07:26:15,2021-12-01 16:45:45
1,JHW,2021-12-01 04:56:00,0.0001,-RA,2021-12-01,2021-12-01 07:26:15,2021-12-01 16:45:45
2,JHW,2021-12-02 00:56:00,0.0001,-RA,2021-12-02,2021-12-02 07:27:17,2021-12-02 16:45:30
3,JHW,2021-12-02 01:56:00,0.0001,-RA,2021-12-02,2021-12-02 07:27:17,2021-12-02 16:45:30
4,JHW,2021-12-02 04:56:00,0.0001,UP BR,2021-12-02,2021-12-02 07:27:17,2021-12-02 16:45:30
...,...,...,...,...,...,...,...
881,JHW,2022-02-27 19:56:00,0.0001,-SN,2022-02-27,2022-02-27 06:55:24,2022-02-27 18:04:26
882,JHW,2022-02-27 20:56:00,0.0001,-SN,2022-02-27,2022-02-27 06:55:24,2022-02-27 18:04:26
883,JHW,2022-02-27 21:56:00,0.0001,-SN,2022-02-27,2022-02-27 06:55:24,2022-02-27 18:04:26
884,JHW,2022-02-27 22:56:00,0.0001,-SN,2022-02-27,2022-02-27 06:55:24,2022-02-27 18:04:26


In [46]:
# merge precip days and sunrise/sunset times on date 
merged3 = pd.merge(precip_days3, sun_times3, on='date')

merged3

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,JHW,2022-12-01 00:56:00,0.0001,-SN BR,2022-12-01,2022-12-01 07:25:59,2022-12-01 16:45:49
1,JHW,2022-12-01 01:56:00,0.0001,-SN,2022-12-01,2022-12-01 07:25:59,2022-12-01 16:45:49
2,JHW,2022-12-01 02:56:00,0.0001,-SN BR,2022-12-01,2022-12-01 07:25:59,2022-12-01 16:45:49
3,JHW,2022-12-01 03:56:00,0.0001,-SN BR,2022-12-01,2022-12-01 07:25:59,2022-12-01 16:45:49
4,JHW,2022-12-01 04:56:00,0.0001,-SN BR,2022-12-01,2022-12-01 07:25:59,2022-12-01 16:45:49
...,...,...,...,...,...,...,...
755,JHW,2023-02-27 19:56:00,0.5100,-RA,2023-02-27,2023-02-27 06:55:46,2023-02-27 18:04:08
756,JHW,2023-02-27 20:56:00,3.8100,UP BR,2023-02-27,2023-02-27 06:55:46,2023-02-27 18:04:08
757,JHW,2023-02-27 21:56:00,2.7900,UP BR,2023-02-27,2023-02-27 06:55:46,2023-02-27 18:04:08
758,JHW,2023-02-27 22:56:00,1.0200,-RA,2023-02-27,2023-02-27 06:55:46,2023-02-27 18:04:08


In [48]:
# filter out only times after sunrise & before sunset 

days1 = merged1[(merged1['datetime'] >= merged1['sunrise']) & (merged1['datetime'] <= merged1['sunset'])]
days2 = merged2[(merged2['datetime'] >= merged2['sunrise']) & (merged2['datetime'] <= merged2['sunset'])]
days3 = merged3[(merged3['datetime'] >= merged3['sunrise']) & (merged3['datetime'] <= merged3['sunset'])]


days1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
2,JHW,2020-12-01 07:56:00,0.0001,-SN BR,2020-12-01,2020-12-01 07:26:30,2020-12-01 16:45:42
3,JHW,2020-12-01 08:56:00,0.0001,-SN BR,2020-12-01,2020-12-01 07:26:30,2020-12-01 16:45:42
4,JHW,2020-12-01 09:56:00,0.0001,-SN,2020-12-01,2020-12-01 07:26:30,2020-12-01 16:45:42
5,JHW,2020-12-01 10:56:00,0.2500,-SN BR,2020-12-01,2020-12-01 07:26:30,2020-12-01 16:45:42
6,JHW,2020-12-01 11:56:00,0.0001,-SN BR,2020-12-01,2020-12-01 07:26:30,2020-12-01 16:45:42
...,...,...,...,...,...,...,...
815,JHW,2021-02-23 15:56:00,0.2500,UP,2021-02-23,2021-02-23 07:01:13,2021-02-23 17:59:50
817,JHW,2021-02-27 10:56:00,0.0001,UP,2021-02-27,2021-02-27 06:55:01,2021-02-27 18:04:44
818,JHW,2021-02-27 12:56:00,0.0001,-RA,2021-02-27,2021-02-27 06:55:01,2021-02-27 18:04:44
819,JHW,2021-02-27 13:56:00,2.7900,-RA,2021-02-27,2021-02-27 06:55:01,2021-02-27 18:04:44


In [50]:
# filter out transition hours
days1['sunrise'] = pd.to_datetime(days1['sunrise'])
days1['sunset'] = pd.to_datetime(days1['sunset'])

days1['sunrise+1'] = days1['sunrise'] + timedelta(hours = 1)
days1['sunset-1'] = days1['sunset'] - timedelta(hours = 1)

days1 = days1[days1['datetime'] <= days1['sunset-1']]
days1 = days1[days1['datetime'] >= days1['sunrise+1']]



days2['sunrise'] = pd.to_datetime(days2['sunrise'])
days2['sunset'] = pd.to_datetime(days2['sunset'])

days2['sunrise+1'] = days2['sunrise'] + timedelta(hours = 1)
days2['sunset-1'] = days2['sunset'] - timedelta(hours = 1)

days2 = days2[days2['datetime'] <= days2['sunset-1']]
days2 = days2[days2['datetime'] >= days2['sunrise+1']]



days3['sunrise'] = pd.to_datetime(days3['sunrise'])
days3['sunset'] = pd.to_datetime(days3['sunset'])

days3['sunrise+1'] = days3['sunrise'] + timedelta(hours = 1)
days3['sunset-1'] = days3['sunset'] - timedelta(hours = 1)

days3 = days3[days3['datetime'] <= days3['sunset-1']]
days3 = days3[days3['datetime'] >= days3['sunrise+1']]

C:\Users\alyha\AppData\Local\Temp\ipykernel_30052\2882998222.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  days1['sunrise'] = pd.to_datetime(days1['sunrise'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_30052\2882998222.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  days1['sunset'] = pd.to_datetime(days1['sunset'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_30052\2882998222.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexe

In [52]:
days1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset,sunrise+1,sunset-1
3,JHW,2020-12-01 08:56:00,0.0001,-SN BR,2020-12-01,2020-12-01 07:26:30,2020-12-01 16:45:42,2020-12-01 08:26:30,2020-12-01 15:45:42
4,JHW,2020-12-01 09:56:00,0.0001,-SN,2020-12-01,2020-12-01 07:26:30,2020-12-01 16:45:42,2020-12-01 08:26:30,2020-12-01 15:45:42
5,JHW,2020-12-01 10:56:00,0.2500,-SN BR,2020-12-01,2020-12-01 07:26:30,2020-12-01 16:45:42,2020-12-01 08:26:30,2020-12-01 15:45:42
6,JHW,2020-12-01 11:56:00,0.0001,-SN BR,2020-12-01,2020-12-01 07:26:30,2020-12-01 16:45:42,2020-12-01 08:26:30,2020-12-01 15:45:42
7,JHW,2020-12-01 12:56:00,0.2500,-SN BR,2020-12-01,2020-12-01 07:26:30,2020-12-01 16:45:42,2020-12-01 08:26:30,2020-12-01 15:45:42
...,...,...,...,...,...,...,...,...,...
815,JHW,2021-02-23 15:56:00,0.2500,UP,2021-02-23,2021-02-23 07:01:13,2021-02-23 17:59:50,2021-02-23 08:01:13,2021-02-23 16:59:50
817,JHW,2021-02-27 10:56:00,0.0001,UP,2021-02-27,2021-02-27 06:55:01,2021-02-27 18:04:44,2021-02-27 07:55:01,2021-02-27 17:04:44
818,JHW,2021-02-27 12:56:00,0.0001,-RA,2021-02-27,2021-02-27 06:55:01,2021-02-27 18:04:44,2021-02-27 07:55:01,2021-02-27 17:04:44
819,JHW,2021-02-27 13:56:00,2.7900,-RA,2021-02-27,2021-02-27 06:55:01,2021-02-27 18:04:44,2021-02-27 07:55:01,2021-02-27 17:04:44


In [54]:
# find total precip over each year
day_total1 = days1['precip(mm)'].sum()
day_total2 = days2['precip(mm)'].sum()
day_total3 = days3['precip(mm)'].sum()

# total number of obs each year
day_rows1 = days1.shape[0]
day_rows2 = days2.shape[0]
day_rows3 = days3.shape[0]

In [56]:
day_precip_per_hour1 = day_total1/ day_rows1
day_precip_per_hour2 = day_total2/ day_rows2
day_precip_per_hour3 = day_total3/ day_rows3


In [58]:
print('2020-2021:', day_precip_per_hour1)
print('2021-2022:', day_precip_per_hour2)
print('2022-2023:', day_precip_per_hour3)

2020-2021: 0.07648920863309351
2021-2022: 0.19567713310580204
2022-2023: 0.20642737226277375


In [60]:
# filter out only times after sunset & before NEXT sunrise 

nights1 = merged1[(merged1['datetime'] < merged1['sunrise']) | (merged1['datetime'] > merged1['sunset'])]
nights2 = merged2[(merged2['datetime'] < merged2['sunrise']) | (merged2['datetime'] > merged2['sunset'])]
nights3 = merged3[(merged3['datetime'] < merged3['sunrise']) | (merged3['datetime'] > merged3['sunset'])]

nights1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,JHW,2020-12-01 00:56:00,0.0001,UP BR,2020-12-01,2020-12-01 07:26:30,2020-12-01 16:45:42
1,JHW,2020-12-01 06:56:00,0.0001,UP,2020-12-01,2020-12-01 07:26:30,2020-12-01 16:45:42
11,JHW,2020-12-01 16:56:00,0.0001,-SN BR,2020-12-01,2020-12-01 07:26:30,2020-12-01 16:45:42
12,JHW,2020-12-01 17:56:00,0.0001,-SN BR,2020-12-01,2020-12-01 07:26:30,2020-12-01 16:45:42
13,JHW,2020-12-01 18:56:00,0.0001,-SN BR,2020-12-01,2020-12-01 07:26:30,2020-12-01 16:45:42
...,...,...,...,...,...,...,...
807,JHW,2021-02-22 19:56:00,0.0001,UP BR,2021-02-22,2021-02-22 07:02:43,2021-02-22 17:58:36
808,JHW,2021-02-22 20:56:00,0.0001,-SN BR,2021-02-22,2021-02-22 07:02:43,2021-02-22 17:58:36
809,JHW,2021-02-22 21:56:00,0.0001,-SN BR,2021-02-22,2021-02-22 07:02:43,2021-02-22 17:58:36
810,JHW,2021-02-22 22:56:00,0.0001,-SN BR,2021-02-22,2021-02-22 07:02:43,2021-02-22 17:58:36


In [62]:
# filter out transition hours
nights1['sunrise'] = pd.to_datetime(nights1['sunrise'])
nights1['sunset'] = pd.to_datetime(nights1['sunset'])

nights1['sunrise-1'] = nights1['sunrise'] - timedelta(hours = 1) + timedelta(days = 1)
nights1['sunset+1'] = nights1['sunset'] + timedelta(hours = 1)

nights1 = nights1[nights1['datetime'] >= nights1['sunset+1']]
nights1 = nights1[nights1['datetime'] <= nights1['sunrise-1']]



nights2['sunrise'] = pd.to_datetime(nights2['sunrise'])
nights2['sunset'] = pd.to_datetime(nights2['sunset'])

nights2['sunrise-1'] = nights2['sunrise'] - timedelta(hours = 1) + timedelta(days = 1)
nights2['sunset+1'] = nights2['sunset'] + timedelta(hours = 1)

nights2 = nights2[nights2['datetime'] >= nights2['sunset+1']]
nights2 = nights2[nights2['datetime'] <= nights2['sunrise-1']]



nights3['sunrise'] = pd.to_datetime(nights3['sunrise'])
nights3['sunset'] = pd.to_datetime(nights3['sunset'])

nights3['sunrise-1'] = nights3['sunrise'] - timedelta(hours = 1) + timedelta(days = 1)
nights3['sunset+1'] = nights3['sunset'] + timedelta(hours = 1)

nights3 = nights3[nights3['datetime'] >= nights3['sunset+1']]
nights3 = nights3[nights3['datetime'] <= nights3['sunrise-1']]


C:\Users\alyha\AppData\Local\Temp\ipykernel_30052\36922422.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nights1['sunrise'] = pd.to_datetime(nights1['sunrise'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_30052\36922422.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nights1['sunset'] = pd.to_datetime(nights1['sunset'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_30052\36922422.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_inde

In [64]:
# find total precip over each year
night_total1 = nights1['precip(mm)'].sum()
night_total2 = nights2['precip(mm)'].sum()
night_total3 = nights3['precip(mm)'].sum()

# total number of obs each year
night_rows1 = nights1.shape[0]
night_rows2 = nights2.shape[0]
night_rows3 = nights3.shape[0]

In [66]:
night_precip_per_hour1 = night_total1/ night_rows1
night_precip_per_hour2 = night_total2/ night_rows2
night_precip_per_hour3 = night_total3/ night_rows3

In [68]:
print('2020-2021:', night_precip_per_hour1)
print('2021-2022:', night_precip_per_hour2)
print('2022-2023:', night_precip_per_hour3)

2020-2021: 0.10660917874396138
2021-2022: 0.1906115555555555
2022-2023: 0.3416450261780105
